In [1]:
import numpy as np
import pandas as pd
import pydicom
%matplotlib inline
import matplotlib.pyplot as plt
import keras 
import cv2

Using TensorFlow backend.


In [2]:
# This function reads in a .dcm file, checks the important fields for our device, and returns a numpy array
# of just the imaging data
def check_dicom(filename): 
    # todo
    
    print('Load file {} ...'.format(filename))
    ds = pydicom.dcmread(filename)     
    try:
        assert(ds.BodyPartExamined=='CHEST')
        assert(ds.Modality=='DX')
        assert(ds.PatientPosition in ['AP', 'PA'])
    except Exception as e:
        print('File {} contains invalid data'.format(filename))
        return None
    img = ds.pixel_array
    return img
    
    
# This function takes the numpy array output by check_dicom and 
# runs the appropriate pre-processing needed for our model input
def preprocess_image(img,img_mean,img_std,img_size): 
    # todo
    proc_img = cv2.resize(img, img_size)
    proc_img = cv2.cvtColor(proc_img, cv2.COLOR_GRAY2RGB)
    proc_img = (proc_img - img_mean)/img_std
    return np.expand_dims(proc_img, 0)

# This function loads in our trained model w/ weights and compiles it 
def load_model(model_path, weight_path):
    with open(model_path, 'r') as json_file:
        model_file = json_file.read()
        model = keras.models.model_from_json(model_file)
        model.load_weights(weight_path)
    
    return model

# This function uses our device's threshold parameters to predict whether or not
# the image shows the presence of pneumonia using our trained model
def predict_image(model, img, thresh): 
    
    return model.predict(img)>thresh 

In [4]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']

model_path = "my_model.json"
weight_path = "xray_class_my_model.best.hdf5"

IMG_SIZE=(224,224) # This might be different if you did not use vgg16

my_model = load_model(model_path, weight_path)
thresh = 0.55

# use the .dcm files to test your prediction
for i in test_dicoms:
    
    img = np.array([])
    img = check_dicom(i)
    if img is None:
        continue
    img_mean = np.mean(img)
    img_std = np.std(img)
        
    img_proc = preprocess_image(img,img_mean,img_std,IMG_SIZE)
    pred = predict_image(my_model,img_proc,thresh)
    print("Pred :" + str(pred[0,0]))

Load file test1.dcm ...
Pred :False
Load file test2.dcm ...
Pred :False
Load file test3.dcm ...
Pred :True
Load file test4.dcm ...
File test4.dcm contains invalid data
Load file test5.dcm ...
File test5.dcm contains invalid data
Load file test6.dcm ...
File test6.dcm contains invalid data
